In [11]:
#Import the data
import pandas as pd
#%pip install openpyxl


# Assuming 'data.xlsx' is your Excel file name in the Downloads folder, adjust the path accordingly
excel_file_path = '/Users/katherinecole/Downloads/Oncology_IO Data.xlsx'

# Read the Excel file into a Pandas dataframe
df = pd.read_excel(excel_file_path)

# Print the first 5 rows of the dataframe
print(df.head())



     ID   Cohort  Year of ICI treatment start Age at ICI start     Sex  \
0  8213  Initial                         2017          61 - 65  Female   
1  8214  Initial                         2018          66 - 70  Female   
2  8215  Initial                         2018          61 - 65  Female   
3  8216  Initial                         2015          56 - 60  Female   
4  8217  Initial                         2018          51 - 55  Female   

   Cancer type   NLR  MSI type  TMB (Mutations/Mb) ICI drug class  \
0   Colorectal  7.60       NaN                 4.9     PD-1/PD-L1   
1  Endometrial  1.19  Unstable                32.5     PD-1/PD-L1   
2         SCLC  1.38    Stable                19.3     PD-1/PD-L1   
3     Melanoma  2.69    Stable                 1.0          Combo   
4        NSCLC  2.54    Stable                10.5     PD-1/PD-L1   

  Best Response Response to ICI Clinical Benefit with ICI Progression  \
0            PD              No                        No         Y

In [12]:
display(df)

,ID,Cohort,Year of ICI treatment start,Age at ICI start,Sex,Cancer type,NLR,MSI type,TMB (Mutations/Mb),ICI drug class,Best Response,Response to ICI,Clinical Benefit with ICI,Progression,Progression-Free Survival (Months),Vital status,Overall Survival (Months),Stage at ICI start,ICI line of treatment,ECOG PS
0,8213,Initial,2017,61 - 65,Female,Colorectal,7.60,NaN,4.9,PD-1/PD-L1,PD,No,No,Yes,0.92,Dead,5.62,IV,Subsequent-line,0.0
1,8214,Initial,2018,66 - 70,Female,Endometrial,1.19,Unstable,32.5,PD-1/PD-L1,PR,Yes,Yes,No,15.64,Alive,15.74,IV,Subsequent-line,1.0
2,8215,Initial,2018,61 - 65,Female,SCLC,1.38,Stable,19.3,PD-1/PD-L1,PR,Yes,Yes,Yes,3.45,Alive,9.59,IV,Subsequent-line,0.0
3,8216,Initial,2015,56 - 60,Female,Melanoma,2.69,Stable,1.0,Combo,PD,No,No,Yes,0.56,Alive,50.14,IV,First-line,0.0
4,8217,Initial,2018,51 - 55,Female,NSCLC,2.54,Stable,10.5,PD-1/PD-L1,PR,Yes,Yes,Yes,4.67,Alive,9.99,IV,Subsequent-line,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2032,10541,Validation,2019,36 - 40,Male,Head & Neck,9.67,Stable,3.5,PD-1/PD-L1,PD,No,No,Yes,2.00,Dead,9.86,IV,Subsequent-line,0.0
2033,10542,Validation,2019,46 - 50,Male,Renal,2.39,Stable,4.4,combo,PD,No,No,Yes,2.63,ALIVE,13.31,IV,First-line,0.0
2034,10543,Validation,2019,41 - 45,Male,Renal,1.46,Stable,2.6,PD-1/PD-L1,SD,No,Yes,Yes,10.48,Dead,10.48,IV,First-line,0.0
2035,10544,Validation,2019,36 - 40,Male,Renal,2.40,Stable,2.6,combo,PD,No,No,Yes,3.22,ALIVE,10.51,III,First-line,0.0


In [10]:
df.describe() #treating all as continuous

,ID,Year of ICI treatment start,NLR,TMB (Mutations/Mb),Progression-Free Survival (Months),Overall Survival (Months),ECOG PS
count,2037.000000,2037.000000,2037.000000,2037.000000,2037.000000,2037.000000,1899.000000
mean,9340.332842,2016.927344,6.308483,9.702946,7.189151,14.047511,0.716166
std,677.656118,1.012022,7.063365,16.582210,9.754267,11.589113,0.662885
min,8212.000000,2014.000000,0.300000,0.000000,0.030000,0.030000,0.000000
25%,8758.000000,2016.000000,2.730000,3.000000,1.450000,4.800000,0.000000
50%,9310.000000,2017.000000,4.310000,5.300000,2.690000,11.370000,1.000000
75%,9901.000000,2018.000000,7.170000,9.800000,8.970000,20.210000,1.000000
max,10545.000000,2019.000000,88.000000,368.600000,67.220000,67.220000,4.000000
